# Learning Notes

GPTモデルの事前学習中に思わぬアクシデントに何回も遭遇したため、<br>
大規模な学習を行う人のために学習上の注意のメモ書きを残しておこうと思いました。<br>
このファイルの内容は[PyTorch Performance Tuning Guide - Szymon Migacz, NVIDIA](https://nvlabs.github.io/eccv2020-mixed-precision-tutorial/files/szymon_migacz-pytorch-performance-tuning-guide.pdf)というNVIDIAの公演スライドと自身の学習経験を元に制作されています。
<br>
内容
- 学習中にGPUのメモリが足りなくなった時の対処法
- 自動混合精度機能を用いた学習の注意点
- その他

In [1]:
import warnings
warnings.simplefilter('ignore')

### 学習中にGPUのメモリが足りなくなった時の対処法

1. メモリをbatch毎に解放する。<br>
batch毎にbatch内部で出てきた変数は消去した方がよい。Pytorch内部の計算グラフが代入によって残ってしまうからか、<br>
Pythonそもそもの仕様なのかはわからないが(詳しい人教えてください)、計算に使用してbatch内部で二度と使わない変数は<br>
削除しないと使えるメモリ領域が減っている。<br>
また、ipynb特有の使用で標準出力されてしまったものはキャッシュされているようである。<br>
学習の際は.ipynbでやるよりも.pyで実行した方が良い。

メモリ解放のやり方は以下の通りである。GPUを使用しているときはGPU上の変数のメモリ領域の解放も忘れてはいけない。

In [2]:
import torch
import gc #メモリ解放を行うライブラリ
device = "cuda" if torch.cuda.is_available() else "cpu"
x = torch.randn(1000,1000).to(device)
del x
gc.collect() #メモリを解放
torch.cuda.empty_cache() #GPUのメモリを解放

2. (CUDAが使える環境限定)自動混合精度機能を使う。<br>
モデルがメモリ不足を引き起こす理由として次に考えられるのは変数がfloat32だからである。<br>
Pytorchではデフォルトの計算でfloat32を使うように指定されているが、深層学習の学習によってはfloat32レベルの精度はいらない可能性がある。<br>
これをfloat16やbfloat16と同時に使用することでメモリ使用量を抑え、さらに計算時間も短縮できる機能がCUDAには盛り込まれている。<br>
Pytorchの公式ドキュメントがこれに関して詳しく書かれているので参考にしてほしい->[torch.cuda.amp](https://pytorch.org/docs/stable/notes/amp_examples.html)

雛形は以下の通りです。(以下のコードは実行できません)

下の自動混合精度機能を用いた計算における追加されたPytorchの機能は以下の二つです。
- torch.cuda.amp.GradScaler
- torch.cuda.amp.autocast<br>
<br>
torch.cuda.amp.GradScalerはfloat32からfloat16やbfloat16にキャストすることで起こるアンダーフローを防止するために損失に大きな数をかけて<br>
勾配を大きくしてこれを防ぎます。損失にかける値のデフォルト値は$2^{16}$となります。<br>
これを実現している箇所がscaler.scale(loss).backward()のところです。<br>
逆伝播が終わってから元の勾配の大きさに戻してパラメーターを更新している箇所がscaler.step(optimizer)になります。<br>
scaler.update()では次のパラメーター更新のためのスケールの更新を行なっています。<br>
<br>
自動混合精度計算を実現している箇所はwith torch.cuda.amp.autocastから始まるブロックの部分になります。<br>
ここの内部では型をキャストしても問題ない計算のみ型をキャストして計算を行なっています。<br>
もちろん深層学習で非常に多く使われる行列計算はキャストの対象になっています。

In [ ]:
model = model().cuda()
optimizer = optim.SGD(model.parameters(), lr)
scaler = torch.cuda.amp.GradScaler(init_scale=2**16, enabled=True)
for epoch in epochs:
    for input, target in data:
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            output = model(input)
            loss = loss_fn(output, target)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

3. batch_sizeを増やすとGPUに変数が乗らない場合は勾配累積を使う<br>
これはbatch毎に勾配を計算してパラメーターを計算するのでは無く、batchの計算では勾配を溜めておいて、最後にパラメーターを更新するというものです。<br>
例えばbatch_sizeが5のデータを6回繰り返せばbatch_sizeが30のデータの勾配を計算することになります。<br>
Pytorchの勾配の仕様として、zero_gradにするまで計算した勾配は加算されていくため、その仕様を利用しましょう。<br>
これにより、自然言語などのタスクでは特定の文章に過剰に適合するなどの問題が起こらなくなります。

実装例としては以下のとおりです。

In [ ]:
model = model().cuda()
optimizer = optim.SGD(model.parameters(), lr)
scaler = torch.cuda.amp.GradScaler(init_scale=2**16, enabled=True)
for epoch in epochs:
    optimizer.zero_grad()
    for input, target in data:
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            output = model(input)
            loss = loss_fn(output, target)
            scaler.scale(loss / len(data)).backward() #勾配は計算しておくだけ、平均しておかないとおかないと損失は大きくなるのでdataのサイズで割る
    scaler.step(optimizer) #最後にパラメーターを更新する。
    scaler.update()

4. 量子化を行う。<br>
モデルのパラメーターfloat32をint8に変換するtorch.qintという型があります。これによりモデルのサイズを減らすことができます。<br>
$\to$[公式ドキュメント](https://pytorch.org/docs/stable/quantization.html)

5. 上4つを試したけど学習がうまくいかない場合<br>
もうどうしようもないので泣きながらモデルの縮小かGPUを増やしましょう。

### 自動混合精度機能を用いた学習の注意点

自動混合精度計算には思わぬ落とし穴があり、それで何時間、何日も学習を無駄にする場合があります。実際の経験からそれらの対処法について解説したいと思います。<br>

1. 絶対にfloat16でキャストをするな。bfloat16でキャストをしろ

非常にシンプルですがこれだけです。理由を説明します。

In [17]:
x = torch.tensor(1e-8)
print("float16: ", x.to(torch.float16))
print("bfloat16: ", x.to(torch.bfloat16))
print("float32: ", x.to(torch.float32))

float16:  tensor(0., dtype=torch.float16)
bfloat16:  tensor(1.0012e-08, dtype=torch.bfloat16)
float32:  tensor(1.0000e-08)


上のコードから、float16は1e-8以下の数字は0にキャストしてしまうことがわかります。<br>
これはゼロ割を含みうる計算の結果がNaNとなるおそれがあり、結果に大きく作用します。<br>

2. 損失がでかすぎてもNaNになるので、torch.cuda.amp.GradScalerの初期化変数のinit_scaleは小さめにとること

初期値は$2^{16}$と非常に大きいです。cos schedulerなどを使っている場合、学習率が大きくなってくると損失が大きくなる恐れがあるので小さめに設定しておきましょう。

一度NaNを含む計算結果で勾配を計算してしまうと全てのパラメーターがNaNとなり、学習のやり直しが発生します。<br>
こうならないためにも学習中のコードでは以下のコードを書いておき、lossがNaNの時に計算をやめるようにしておくことをおすすめします。<br>

In [ ]:
if torch.isnan(loss.detach()):
    print("Loss is NaN.")
    break

### その他

#### 学習の高速化

1. モデルをコンパイルする<br>
モデルをtorch.compileでコンパイルすることで計算の効率化、最適化を行ってくれるものになります。使わない手はないでしょう。

2. torch.backendsをいじる<br>
- torch.backends.cudnn.benchmark<br>torch.backends.cudnn.benchmarkをTrueにすることでネットワークの構成に対して最適なアルゴリズムを見つけて計算を行うため、計算が早くなります。<br>(しかし、再現性はなくなります。)
- torch.backends.cudnn.allow_tf32<br>
torch.backends.cudnn.allow_tf32をTrueにすることで対応しているGPUならばTensorFloat32コアを使用して計算を行います。<br>
他にもtorch.backendsには計算の最適化を行う機能があるので調べてみると良いです。